In [21]:
from utils import *
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

import numpy as np
import torch

import matplotlib.pyplot as plt

if torch.cuda.is_available():
    torch.cuda.set_device(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
def load_data(base_path="./data"):
    """ Load the data in PyTorch Tensor.

    :return: (zero_train_matrix, train_data, valid_data, test_data)
        WHERE:
        zero_train_matrix: 2D sparse matrix where missing entries are
        filled with 0.
        train_data: 2D sparse matrix
        valid_data: A dictionary {user_id: list,
        user_id: list, is_correct: list}
        test_data: A dictionary {user_id: list,
        user_id: list, is_correct: list}
    """
    train_matrix = load_train_sparse(base_path).toarray()
    valid_data = load_valid_csv(base_path)
    test_data = load_public_test_csv(base_path)

    zero_train_matrix = train_matrix.copy()
    # Fill in the missing entries to 0.
    zero_train_matrix[np.isnan(train_matrix)] = 0
    # Change to Float Tensor for PyTorch.
    zero_train_matrix = torch.FloatTensor(zero_train_matrix)
    train_matrix = torch.FloatTensor(train_matrix)

    return zero_train_matrix, train_matrix, valid_data, test_data


class AutoEncoder(nn.Module):
    def __init__(self, num_question, k=100):
        """ Initialize a class AutoEncoder.

        :param num_question: int
        :param k: int
        """
        super(AutoEncoder, self).__init__()

        # Define linear functions.
        self.g = nn.Linear(num_question, k)
        self.h = nn.Linear(k, num_question)

    def get_weight_norm(self):
        """ Return ||W^1||^2 + ||W^2||^2.

        :return: float
        """
        g_w_norm = torch.norm(self.g.weight, 2) ** 2
        h_w_norm = torch.norm(self.h.weight, 2) ** 2
        return g_w_norm + h_w_norm

    def forward(self, inputs):
        """ Return a forward pass given inputs.

        :param inputs: user vector.
        :return: user vector.
        """
        #####################################################################
        # TODO:                                                             #
        # Implement the function as described in the docstring.             #
        # Use sigmoid activations for f and g.                              #
        #####################################################################
        out = torch.sigmoid(self.g(inputs))
        out = torch.sigmoid(self.h(out))
        #####################################################################
        #                       END OF YOUR CODE                            #
        #####################################################################
        return out


def train(model, lr, lamb, train_data, zero_train_data, valid_data, num_epoch):
    """ Train the neural network, where the objective also includes
    a regularizer.

    :param model: Module
    :param lr: float
    :param lamb: float
    :param train_data: 2D FloatTensor
    :param zero_train_data: 2D FloatTensor
    :param valid_data: Dict
    :param num_epoch: int
    :return: None
    """
    # TODO: Add a regularizer to the cost function. 
    
    # Tell PyTorch you are training the model.
    model.train()

    # Define optimizers and loss function.
    optimizer = optim.SGD(model.parameters(), lr=lr)
    num_student = train_data.shape[0]

    # For plotting
    train_losses = []
    valid_accuracies = []
    epochs = []

    for epoch in range(0, num_epoch):
        train_loss = 0.

        for user_id in range(num_student):
            inputs = Variable(zero_train_data[user_id]).unsqueeze(0).to(device)
            target = inputs.clone().to(device)

            optimizer.zero_grad()
            output = model(inputs)

            # Mask the target to only compute the gradient of valid entries.
            nan_mask = np.isnan(train_data[user_id].unsqueeze(0).numpy())
            target[0][nan_mask] = output[0][nan_mask]

            loss = torch.sum((output - target) ** 2.) + lamb/2*model.get_weight_norm()
            loss.backward()

            train_loss += loss.item()
            optimizer.step()

        valid_acc = evaluate(model, zero_train_data, valid_data)
        print("Epoch: {} \tTraining Cost: {:.6f} \tValid Acc: {}".format(epoch, train_loss, valid_acc))
        
        train_losses.append(train_loss)
        valid_accuracies.append(valid_acc)
        epochs.append(epoch)

    # Plot training curve
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)

    ax1.plot(epochs, train_losses)
    ax1.set_title("Training Loss Curve")
    ax1.set(ylabel='Loss')

    ax2.plot(epochs, valid_accuracies)
    ax2.set_title("Validation Accuracy Curve")
    ax2.set(xlabel='Epochs', ylabel='Accuracy')

    plt.subplots_adjust(hspace=0.5)
    plt.show()

    #####################################################################
    #                       END OF YOUR CODE                            #
    #####################################################################


def evaluate(model, train_data, valid_data):
    """ Evaluate the valid_data on the current model.

    :param model: Module
    :param train_data: 2D FloatTensor
    :param valid_data: A dictionary {user_id: list,
    question_id: list, is_correct: list}
    :return: float
    """
    # Tell PyTorch you are evaluating the model.
    model.eval()

    total = 0
    correct = 0

    for i, u in enumerate(valid_data["user_id"]):
        inputs = Variable(train_data[u]).unsqueeze(0).to(device)
        output = model(inputs)

        # Accuracy
        guess = output[0][valid_data["question_id"][i]].item() >= 0.5
        if guess == valid_data["is_correct"][i]:
            correct += 1
        total += 1

    return correct / float(total)

In [3]:
zero_train_matrix, train_matrix, valid_data, test_data = load_data()

#####################################################################
# TODO:                                                             #
# Try out 5 different k and select the best k using the             #
# validation set.                                                   #
#####################################################################


# Set model hyperparameters.
k = 50
model = AutoEncoder(train_matrix.shape[1], k)
model.to(device)

# Set optimization hyperparameters.
lr = 0.01
num_epoch = 50
lamb = 0

train(model, lr, lamb, train_matrix, zero_train_matrix, valid_data, num_epoch)
#####################################################################
#                       END OF YOUR CODE                            #
#####################################################################

Epoch: 0 	Training Cost: 13804.152070 	Valid Acc: 0.6081004798193621
Epoch: 1 	Training Cost: 12773.334682 	Valid Acc: 0.6234829240756421
Epoch: 2 	Training Cost: 12419.077636 	Valid Acc: 0.6253175275190517
Epoch: 3 	Training Cost: 12256.886973 	Valid Acc: 0.6271521309624611
Epoch: 4 	Training Cost: 12121.042528 	Valid Acc: 0.6279988710132656
Epoch: 5 	Training Cost: 11987.572649 	Valid Acc: 0.630962461191081


KeyboardInterrupt: 

In [5]:
test_acc = evaluate(model, zero_train_matrix, test_data)
print(test_acc)

0.6917866215071973


In [4]:
#torch.save(model.state_dict(), './model.pt')

model = AutoEncoder(train_matrix.shape[1], k)
model.load_state_dict(torch.load('./model.pt'))
model.eval()
model.to(device)

AutoEncoder(
  (g): Linear(in_features=1774, out_features=50, bias=True)
  (h): Linear(in_features=50, out_features=1774, bias=True)
)